In [ ]:
### Recommendation System using Aprioti Algo (Purchase Recommendation System)
### __Apriori Algorithm__
# Ref: DeepSeek ai


import itertools

def apriori(transactions, min_support, max_length=None):
    transactions = [set(t) for t in transactions]
    num_transactions = len(transactions)
    frequent_itemsets = {}

    # Generate frequent 1-itemsets
    item_counts = {}
    for t in transactions:
        for item in t:
            item_counts[item] = item_counts.get(item, 0) + 1

    frequent_itemsets[1] = {
        frozenset([item]): count / num_transactions
        for item, count in item_counts.items()
        if count / num_transactions >= min_support
    }

    # Generate frequent k-itemsets
    k = 2
    while True:
        if max_length and k > max_length:
            break

        # Generate candidate itemsets
        candidates = set()
        prev_items = [item for itemset in frequent_itemsets[k-1] for item in itemset]
        unique_items = list(set(prev_items))

        # Create combinations and check subsets
        for candidate in itertools.combinations(unique_items, k):
            candidate = frozenset(candidate)
            valid = True
            for subset in itertools.combinations(candidate, k-1):
                if frozenset(subset) not in frequent_itemsets[k-1]:
                    valid = False
                    break
            if valid:
                candidates.add(candidate)

        # Calculate support for candidates
        candidate_counts = {}
        for candidate in candidates:
            count = sum(1 for t in transactions if candidate.issubset(t))
            support = count / num_transactions
            if support >= min_support:
                candidate_counts[candidate] = support

        if not candidate_counts:
            break

        frequent_itemsets[k] = candidate_counts
        k += 1

    return frequent_itemsets

def generate_rules(frequent_itemsets, min_confidence):
    rules = []
    for k in frequent_itemsets:
        if k < 2:
            continue

        for itemset, support in frequent_itemsets[k].items():
            # Generate all possible antecedents
            for i in range(1, k):
                for antecedent in itertools.combinations(itemset, i):
                    antecedent = frozenset(antecedent)
                    consequent = itemset - antecedent

                    if len(consequent) == 0:
                        continue

                    # Get antecedent support
                    ant_support = frequent_itemsets[len(antecedent)].get(antecedent, 0)
                    if ant_support == 0:
                        continue

                    confidence = support / ant_support
                    if confidence >= min_confidence:
                        rules.append((antecedent, consequent, support, confidence))

    return rules

def recommend(items, rules, max_recommendations=None):
    items = set(items)
    recommendations = {}

    for antecedent, consequent, support, confidence in rules:
        if antecedent.issubset(items):
            for item in consequent:
                if item not in items:
                    if item not in recommendations or confidence > recommendations[item][0]:
                        recommendations[item] = (confidence, support)

    # Sort by confidence (descending), then support (descending)
    sorted_recs = sorted(recommendations.items(), key=lambda x: (-x[1][0], -x[1][1]))
    return [item for item, _ in sorted_recs[:max_recommendations]]

# Example usage
transactions = [
    ['milk', 'bread', 'eggs'],
    ['milk', 'bread'],
    ['bread', 'eggs'],
    ['milk', 'eggs'],
    # ['milk', 'eggs'],
    ['bread', 'milk']
]

min_support = 0.4
min_confidence = 0.7

# Generate frequent itemsets
frequent_itemsets = apriori(transactions, min_support)

# Generate association rules
rules = generate_rules(frequent_itemsets, min_confidence)

# Get recommendations
test_cases = [
    ['milk'],
    ['bread'],
    ['eggs'],
    ['milk', 'bread']
]

for items in test_cases:
    print(f"Recommendations for {items}:")
    print(recommend(items, rules))
    print()


Recommendations for ['milk']:
['bread']

Recommendations for ['bread']:
['milk']

Recommendations for ['eggs']:
[]

Recommendations for ['milk', 'bread']:
[]

